In [1]:
%matplotlib inline

import os
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy import stats

import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse
from sklearn.preprocessing import MinMaxScaler

plt.style.use('ggplot')

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.5/dist-packages/pmdarima/__init__.py:100: UserWarning: pmdarima is not built or tested against versions of python older than 3.6. Your python version (3.5.2) is not guaranteed to be supported
  _py_version.major, _py_version.minor, _py_version.micro,


Ingest Data File and Make Necessary Adjustments

In [83]:
data = pd.read_csv('cap3_data.csv')

In [84]:
data = data.drop(['Unnamed: 0'], axis=1)
data['UTC Time at End of Hour'] = pd.to_datetime(data['UTC Time at End of Hour'], errors='raise')
data = data.set_index('UTC Time at End of Hour')

Calculate Demand Mean and Fill NaNs in Column with Mean/Zeros

In [6]:
og_mean = data['Demand (MW)'].mean()
og_mean

343.39225844920753

In [29]:
zero_mean = data['Demand (MW)'].mean()
zero_mean

85.02272118128093

In [85]:
data['Demand (MW)'] = data['Demand (MW)'].fillna(0)
#data['Demand (MW)'] = data['Demand (MW)'].fillna(og_mean)

In [86]:
data.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3156438 entries, 2015-07-01 06:00:00 to 2021-01-01 05:00:00
Data columns (total 13 columns):
Balancing Authority                                       3156438 non-null object
Demand Forecast (MW)                                      745868 non-null float64
Demand (MW)                                               3156438 non-null float64
Net Generation (MW)                                       1278726 non-null float64
Total Interchange (MW)                                    2342764 non-null float64
Net Generation (MW) from Coal                             2198539 non-null float64
Net Generation (MW) from Natural Gas                      2382768 non-null float64
Net Generation (MW) from Nuclear                          1808818 non-null float64
Net Generation (MW) from All Petroleum Products           2182327 non-null float64
Net Generation (MW) from Hydropower and Pumped Storage    2511687 non-null float64
Net Generation (MW) from S

In [9]:
data.head()

,Balancing Authority,Demand Forecast (MW),Demand (MW),Net Generation (MW),Total Interchange (MW),Net Generation (MW) from Coal,Net Generation (MW) from Natural Gas,Net Generation (MW) from Nuclear,Net Generation (MW) from All Petroleum Products,Net Generation (MW) from Hydropower and Pumped Storage,Net Generation (MW) from Solar,Net Generation (MW) from Wind,Net Generation (MW) from Other Fuel Sources
UTC Time at End of Hour,,,,,,,,,,,,,
2015-07-01 06:00:00,AEC,882.0,422.0,670.0,248.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-07-01 07:00:00,AEC,819.0,395.0,620.0,225.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-07-01 08:00:00,AEC,782.0,382.0,637.0,255.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-07-01 09:00:00,AEC,763.0,370.0,619.0,249.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-07-01 10:00:00,AEC,774.0,383.0,633.0,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
data

,Balancing Authority,Demand Forecast (MW),Demand (MW),Net Generation (MW),Total Interchange (MW),Net Generation (MW) from Coal,Net Generation (MW) from Natural Gas,Net Generation (MW) from Nuclear,Net Generation (MW) from All Petroleum Products,Net Generation (MW) from Hydropower and Pumped Storage,Net Generation (MW) from Solar,Net Generation (MW) from Wind,Net Generation (MW) from Other Fuel Sources,Demand Shift
UTC Time at End of Hour,,,,,,,,,,,,,,
2015-07-01 06:00:00,AEC,882.0,422.0,670.0,248.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,395.0
2015-07-01 07:00:00,AEC,819.0,395.0,620.0,225.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,382.0
2015-07-01 08:00:00,AEC,782.0,382.0,637.0,255.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,370.0
2015-07-01 09:00:00,AEC,763.0,370.0,619.0,249.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,383.0
2015-07-01 10:00:00,AEC,774.0,383.0,633.0,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,397.0
2015-07-01 11:00:00,AEC,810.0,397.0,617.0,220.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,420.0
2015-07-01 12:00:00,AEC,867.0,420.0,619.0,199.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,455.0
2015-07-01 13:00:00,AEC,940.0,455.0,680.0,225.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,493.0
2015-07-01 14:00:00,AEC,NaN,493.0,695.0,202.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,554.0


In [87]:
demand = data['Demand (MW)']

In [88]:
demand_shift = data.groupby('Balancing Authority')['Demand (MW)'].shift(periods=-1)

In [94]:
data['Demand Shift'] = demand_shift

In [105]:
shifted_data = data.dropna(subset=['Demand Shift'])

In [108]:
shifted_data

In [109]:
size = int(len(shifted_data) * (2/3))
train_data_shift = shifted_data[:size]
test_data_shift = shifted_data[size:]

Establishing Baseline Model

In [112]:
arima_rmse_error_shift = rmse(test_data_shift['Demand (MW)'], test_data_shift['Demand Shift'])
arima_mse_error_shift = arima_rmse_error**2
mean_value_shift = data['Demand (MW)'].mean()

In [113]:
print('MSE Error: ', arima_mse_error_shift)
print('RMSE Error: ', arima_rmse_error_shift)
print('Mean: ', mean_value_shift)

MSE Error:  nan
RMSE Error:  66.89483074455755
Mean:  85.02272118128093


In [93]:
print(data.isna().sum())

Balancing Authority                                             0
Demand Forecast (MW)                                      2410570
Demand (MW)                                                     0
Net Generation (MW)                                       1877712
Total Interchange (MW)                                     813674
Net Generation (MW) from Coal                              957899
Net Generation (MW) from Natural Gas                       773670
Net Generation (MW) from Nuclear                          1347620
Net Generation (MW) from All Petroleum Products            974111
Net Generation (MW) from Hydropower and Pumped Storage     644751
Net Generation (MW) from Solar                             593288
Net Generation (MW) from Wind                              909764
Net Generation (MW) from Other Fuel Sources                723462
dtype: int64


In [67]:
demand_diff.head()

UTC Time at End of Hour
2015-07-01 06:00:00     NaN
2015-07-01 07:00:00   -27.0
2015-07-01 08:00:00   -13.0
2015-07-01 09:00:00   -12.0
2015-07-01 10:00:00    13.0
Name: Demand (MW), dtype: float64

In [ ]:
#auto_arima(data['Demand (MW)'], 
#           seasonal=True, 
#           m=12, 
#           max_p=7, 
#           max_d=5, 
#           max_q=7, 
#           max_P=4, 
#           max_D=4, 
#           max_Q=4).summary()

Defining the Size of Training and Testing Data

In [31]:
size = int(len(demand) * (2/3))
train_data = demand[:size]
test_data = demand[size:]

In [ ]:
# walk-forward validation
for t in range(len(test)):
	model = ARIMA(history, order=(5,1,0))
	model_fit = model.fit()
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = test[t]
	history.append(obs)
	print('predicted=%f, expected=%f' % (yhat, obs))

Creating the ARIMA Model

In [32]:
arima_model = SARIMAX(train_data)#, order = (2,1,1), seasonal_order = (4,0,3,12))
arima_result = arima_model.fit()
arima_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:            Demand (MW)   No. Observations:              2104292
Model:               SARIMAX(1, 0, 0)   Log Likelihood            -2527540.725
Date:                Thu, 14 Jan 2021   AIC                        5055085.451
Time:                        23:00:53   BIC                        5055110.570
Sample:                             0   HQIC                       5055092.164
                            - 2104292                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.9987   5.42e-05   1.84e+04      0.000       0.999       0.999
sigma2       701.3899      0.109   6408.257      0.000     701.175     701.604
===================================================================================
Ljung-Box (Q):                  3008112.60   Jarque-Bera (JB):      156548511240.58
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.40   Skew:                             1.19
Prob(H) (two-sided):                  0.00   Kurtosis:                      1339.22
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

Baseline Predictions with Nothing Imputed for NaNs

In [33]:
arima_base_pred = arima_result.predict(
    start = len(train_data), 
    end = (len(train_data) + len(test_data)-1), typ="levels").rename("ARIMA Baseline Predictions")

In [ ]:
arima_base_pred

Predictions with Zeroes for NaNs

In [33]:
arima_zero_pred = arima_result.predict(
    start = len(train_data), 
    end = (len(train_data) + len(test_data)-1), typ="levels").rename("ARIMA Zero Predictions")

In [34]:
arima_zero_pred

2104292   -5.434722e-323
2104293   -5.434722e-323
2104294   -5.434722e-323
2104295   -5.434722e-323
2104296   -5.434722e-323
               ...      
3156433   -5.434722e-323
3156434   -5.434722e-323
3156435   -5.434722e-323
3156436   -5.434722e-323
3156437   -5.434722e-323
Name: ARIMA Zero Predictions, Length: 1052146, dtype: float64

Predictions with Demand Mean

In [20]:
arima_mean_pred = arima_result.predict(
    start = len(train_data), 
    end = (len(train_data) + len(test_data)-1), typ="levels").rename("ARIMA Mean Predictions")

In [21]:
arima_mean_pred

2104292     3.409387e+02
2104293     3.385028e+02
2104294     3.360842e+02
2104295     3.336829e+02
2104296     3.312988e+02
               ...      
3156433    3.409053e-322
3156434    3.409053e-322
3156435    3.409053e-322
3156436    3.409053e-322
3156437    3.409053e-322
Name: ARIMA Predictions, Length: 1052146, dtype: float64

Measuring Performance of the ARIMA Model

Performance when No Data is Imputed for the NaNs

In [50]:
arima_rmse_error = rmse(test_data, arima_base_pred)
arima_mse_error = arima_rmse_error**2
mean_value = data['Demand (MW)'].mean()

#print(f'MSE Error: {arima_mse_error}\nRMSE Error: {arima_rmse_error}\nMean: {mean_value}')

In [51]:
print('MSE Error: ', arima_mse_error)
print('RMSE Error: ', arima_rmse_error)
print('Mean: ', mean_value)

MSE Error:  nan
RMSE Error:  nan
Mean:  343.39225844920753


Performance when the NaNs are Filled with Zeroes

In [48]:
arima_rmse_error = rmse(test_data, arima_zero_pred)
arima_mse_error = arima_rmse_error**2
mean_value = data['Demand (MW)'].mean()

print(f'MSE Error: {arima_mse_error}\nRMSE Error: {arima_rmse_error}\nMean: {mean_value}')

MSE Error: 48550.670196955056
RMSE Error: 220.34216618013687
Mean: 85.02272118128093


Performance when NaNs are Filled with the Mean Demand

In [22]:
arima_rmse_error = rmse(test_data, arima_mean_pred)
arima_mse_error = arima_rmse_error**2
mean_value = data['Demand (MW)'].mean()

print(f'MSE Error: {arima_mse_error}\nRMSE Error: {arima_rmse_error}\nMean: {mean_value}')

MSE Error: 137802.64675154808
RMSE Error: 371.2177888403896
Mean: 343.3922584492074


LSTM Model

In [5]:
import tensorflow as tf

In [6]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

Instantiating the Scaler and Scaling the Data

In [7]:
scaler = MinMaxScaler()

In [35]:
type(train_data)

pandas.core.series.Series

In [58]:
short_lstm_train = demand[-150:-10]
short_lstm_test = demand[-10:]

In [59]:
short_lstm_train.value_counts

<bound method IndexOpsMixin.value_counts of UTC Time at End of Hour
2020-12-26 00:00:00   NaN
2020-12-26 01:00:00   NaN
2020-12-26 02:00:00   NaN
2020-12-26 03:00:00   NaN
2020-12-26 04:00:00   NaN
2020-12-26 05:00:00   NaN
2020-12-26 06:00:00   NaN
2020-12-26 07:00:00   NaN
2020-12-26 08:00:00   NaN
2020-12-26 09:00:00   NaN
2020-12-26 10:00:00   NaN
2020-12-26 11:00:00   NaN
2020-12-26 12:00:00   NaN
2020-12-26 13:00:00   NaN
2020-12-26 14:00:00   NaN
2020-12-26 15:00:00   NaN
2020-12-26 16:00:00   NaN
2020-12-26 17:00:00   NaN
2020-12-26 18:00:00   NaN
2020-12-26 19:00:00   NaN
2020-12-26 20:00:00   NaN
2020-12-26 21:00:00   NaN
2020-12-26 22:00:00   NaN
2020-12-26 23:00:00   NaN
2020-12-27 00:00:00   NaN
2020-12-27 01:00:00   NaN
2020-12-27 02:00:00   NaN
2020-12-27 03:00:00   NaN
2020-12-27 04:00:00   NaN
2020-12-27 05:00:00   NaN
                       ..
2020-12-30 14:00:00   NaN
2020-12-30 15:00:00   NaN
2020-12-30 16:00:00   NaN
2020-12-30 17:00:00   NaN
2020-12-30 18:00:00   

In [37]:
train_lstm = short_lstm_train.values.reshape(-1, 1)
test_lstm = short_lstm_test.values.reshape(-1, 1)

In [ ]:
x_train, x_test, y_train, y_test

In [38]:
scaler.fit(train_lstm)
scaled_train = scaler.transform(train_lstm)
scaled_test = scaler.transform(test_lstm)

Instantiating the LSTM Model

In [39]:
n_input = 12
n_features = 1
num_pred = 1
generator = TimeseriesGenerator(scaled_train, 
                                train_lstm, 
                                length=n_input, batch_size=1)

In [40]:
lstm_model = Sequential()
lstm_model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
lstm_model.add(Dense(num_pred, activation='linear'))
lstm_model.compile(optimizer='adam', loss='mse')

lstm_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
unified_lstm_1 (UnifiedLSTM) (None, 200)               161600    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 161,801
Trainable params: 161,801
Non-trainable params: 0
_________________________________________________________________


Verifying Scaled/Train Data

In [41]:
batch_0 = generator[0]
x, y = batch_0
x

array([[[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]]])

In [42]:
y

array([[nan]])

In [26]:
lstm_model.fit_generator(generator, epochs=2, verbose=1)

Epoch 1/2
 262015/2104280 [==>...........................] - ETA: 23:22:36 - loss: 249803.8249

KeyboardInterrupt: 

In [ ]:
lstm_predictions_scaled = list()

batch = scaled_train_data[-n_input:]
current_batch = batch.reshape((1, n_input, n_features))

In [ ]:
for i in range(len(test_data)):
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_predictions_scaled.append(lstm_pred)
    current_batch = np.append(current_batch[:,1:,:],[[lstm_pred]], axis=1)

In [ ]:
lstm_predictions_scaled

In [ ]:
lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)
lstm_predictions

In [ ]:
test_data['LSTM_Predictions'] = lstm_predictions
test_data

In [ ]:
test_data.plot(figsize=(16,5), legend=True)
test_data['LSTM_Predictions'].plot(legend=True)

In [ ]:
lstm_rmse_error = rmse(test_data['Demand (MW)'],
                      test_data['LSTM_Predictions'])
lstm_mst_error = lstm_rmse_error**2
mean_value = data['Demand (MW)'].mean()

print(f'MSE Error: {lstm_mse_error}\n
RMSE Error:{lstm_rmse_error}\
nMean: {mean_value})

In [8]:
def windowize_data(data, n_prev):
    n_predictions = len(data) - n_prev
    y = data[n_prev:]
    # this might be too clever
    indices = np.arange(n_prev) + np.arange(n_predictions)[:, None]
    x = data[indices, None]
    return x, y

In [9]:
def split_and_windowize(data, n_prev, fraction_test=0.3):
    n_predictions = len(data) - 2*n_prev
    
    n_test  = int(fraction_test * n_predictions)
    n_train = n_predictions - n_test   
    
    x_train, y_train = windowize_data(data[:n_train], n_prev)
    x_test, y_test = windowize_data(data[n_train:], n_prev)
    return x_train, x_test, y_train, y_test

In [10]:
n_prev = 50

In [12]:
x_train, x_test, y_train, y_test = split_and_windowize(demand, n_prev=50)

In [17]:
#x_train, x_test, y_train, y_test = split_and_windowize(sin_t_noisy, n_prev)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2209387, 50, 1), (946951, 50, 1), (2209387,), (946951,))

In [ ]:
model = keras.Sequential()
model.add(keras.layers.SimpleRNN(32, input_shape=(n_prev, 1), return_sequences=True))
model.add(keras.layers.SimpleRNN(32, return_sequences=False))
model.add(keras.layers.Dense(1, activation='linear'))
model.compile(optimizer='rmsprop',
              loss='mse')

In [ ]:
model.summary()

In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=2)

In [ ]:
y_pred = model.predict(x_test)